In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, \
    Conv2DTranspose, BatchNormalization, Dropout, LeakyReLU, Add
from tensorflow.keras.utils import plot_model


def convolution_block(x, filters, size, strides=(1, 1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation:
        x = LeakyReLU(alpha=0.1)(x)
    return x


def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3, 3))
    x = convolution_block(x, num_filters, (3, 3), activation=False)
    x = Add()([x, blockInput])
    return x

2022-06-08 10:00:13.661727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nathan/sandbox/elastix/elastix-5.0.1-linux/lib
2022-06-08 10:00:13.661751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def unet_model(n_classes=1, im_sz=224, n_channels=3, n_filters_start=32, growth_factor=2, upconv=True):
    droprate=0.1
    n_filters = n_filters_start
    inputs = Input((im_sz, im_sz, n_channels))

    conv1 = convolution_block(inputs, n_filters, (3, 3))
    conv1 = residual_block(conv1, n_filters)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(droprate)(pool1)

    n_filters *= growth_factor
    pool1 = BatchNormalization()(pool1)
    conv2 = convolution_block(pool1, n_filters, (3, 3))
    conv2 = residual_block(conv2, n_filters)
    conv2 = residual_block(conv2, n_filters)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    n_filters *= growth_factor
    pool2 = BatchNormalization()(pool2)
    pool2 = convolution_block(pool2, n_filters, (3, 3))
    conv3 = residual_block(pool2, n_filters)
    conv3 = residual_block(conv3, n_filters)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    n_filters *= growth_factor
    pool3 = BatchNormalization()(pool3)
    pool3 = convolution_block(pool3, n_filters, (3, 3))
    conv4_0 = residual_block(pool3, n_filters)
    conv4_0 = residual_block(conv4_0, n_filters)
    pool4_0 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_0 = Dropout(droprate)(pool4_0)

    n_filters *= growth_factor
    pool4_0 = BatchNormalization()(pool4_0)
    pool4_0 = convolution_block(pool4_0, n_filters, (3, 3))
    conv4_1 = residual_block(pool4_0, n_filters)
    conv4_1 = residual_block(conv4_1, n_filters)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_1 = Dropout(droprate)(pool4_1)

    n_filters *= growth_factor
    pool4_2 = BatchNormalization()(pool4_1)
    pool4_2 = convolution_block(pool4_2, n_filters, (3, 3))
    conv5 = residual_block(pool4_2, n_filters)
    conv5 = residual_block(conv5, n_filters)
    conv5 = Dropout(droprate)(conv5)

    n_filters //= growth_factor
    if upconv:
        up6_1 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv5), conv4_1])
    else:
        up6_1 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4_1])
    up6_1 = BatchNormalization()(up6_1)
    up6_1 = convolution_block(up6_1, n_filters, (3, 3))
    conv6_1 = residual_block(up6_1, n_filters)
    conv6_1 = residual_block(conv6_1, n_filters)
    conv6_1 = Dropout(droprate)(conv6_1)

    n_filters //= growth_factor
    if upconv:
        up6_2 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_1), conv4_0])
    else:
        up6_2 = concatenate([UpSampling2D(size=(2, 2))(conv6_1), conv4_0])
    up6_2 = BatchNormalization()(up6_2)
    up6_2 = convolution_block(up6_2, n_filters, (3, 3))
    conv6_2 = residual_block(up6_2, n_filters)
    conv6_2 = residual_block(conv6_2, n_filters)
    conv6_2 = Dropout(droprate)(conv6_2)

    n_filters //= growth_factor
    if upconv:
        up7 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_2), conv3])
    else:
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6_2), conv3])
    up7 = BatchNormalization()(up7)
    up7 = convolution_block(up7, n_filters, (3, 3))
    conv7 = residual_block(up7, n_filters)
    conv7 = residual_block(conv7, n_filters)
    conv7 = Dropout(droprate)(conv7)

    n_filters //= growth_factor
    if upconv:
        up8 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    else:
        up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2])
    up8 = BatchNormalization()(up8)
    up8 = convolution_block(up8, n_filters, (3, 3))
    conv8 = residual_block(up8, n_filters)
    conv8 = residual_block(conv8, n_filters)
    conv8 = Dropout(droprate)(conv8)

    n_filters //= growth_factor
    if upconv:
        up9 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    else:
        up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1])
    up9 = BatchNormalization()(up9)
    up9 = convolution_block(up9, n_filters, (3, 3))
    conv9 = residual_block(up9, n_filters)
    conv9 = residual_block(conv9, n_filters)

    conv10 = Conv2D(n_classes, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [3]:
def get_lowres_encoder(im_sz=224, n_channels=3, n_filters_start=32, growth_factor=2):
    droprate=0.1
    n_filters = n_filters_start
    inputs = Input((im_sz, im_sz, n_channels))

    conv1 = convolution_block(inputs, n_filters, (3, 3))
    conv1 = residual_block(conv1, n_filters)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(droprate)(pool1)

    n_filters *= growth_factor
    pool1 = BatchNormalization()(pool1)
    conv2 = convolution_block(pool1, n_filters, (3, 3))
    conv2 = residual_block(conv2, n_filters)
    conv2 = residual_block(conv2, n_filters)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    n_filters *= growth_factor
    pool2 = BatchNormalization()(pool2)
    pool2 = convolution_block(pool2, n_filters, (3, 3))
    conv3 = residual_block(pool2, n_filters)
    conv3 = residual_block(conv3, n_filters)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    n_filters *= growth_factor
    pool3 = BatchNormalization()(pool3)
    pool3 = convolution_block(pool3, n_filters, (3, 3))
    conv4_0 = residual_block(pool3, n_filters)
    conv4_0 = residual_block(conv4_0, n_filters)
    pool4_0 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_0 = Dropout(droprate)(pool4_0)

    n_filters *= growth_factor
    pool4_0 = BatchNormalization()(pool4_0)
    pool4_0 = convolution_block(pool4_0, n_filters, (3, 3))
    conv4_1 = residual_block(pool4_0, n_filters)
    conv4_1 = residual_block(conv4_1, n_filters)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_1 = Dropout(droprate)(pool4_1)

    n_filters *= growth_factor
    pool4_2 = BatchNormalization()(pool4_1)
    pool4_2 = convolution_block(pool4_2, n_filters, (3, 3))
    conv5 = residual_block(pool4_2, n_filters)
    conv5 = residual_block(conv5, n_filters)
    conv5 = Dropout(droprate)(conv5)
    
    model = Model(inputs=inputs, outputs=conv5)
    return model

In [4]:
def bi_res_unet(n_classes=1, im_sz=224, n_channels=3, n_filters_start=32, growth_factor=2, upconv=True):
    droprate=0.1
    n_filters = n_filters_start
    lowres_encoder = get_lowres_encoder(
        im_sz=im_sz, n_channels=n_channels, n_filters_start=n_filters_start,
        growth_factor=growth_factor
    )
    inputs = Input((im_sz, im_sz, n_channels))

    conv1 = convolution_block(inputs, n_filters, (3, 3))
    conv1 = residual_block(conv1, n_filters)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(droprate)(pool1)

    n_filters *= growth_factor
    pool1 = BatchNormalization()(pool1)
    conv2 = convolution_block(pool1, n_filters, (3, 3))
    conv2 = residual_block(conv2, n_filters)
    conv2 = residual_block(conv2, n_filters)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    n_filters *= growth_factor
    pool2 = BatchNormalization()(pool2)
    pool2 = convolution_block(pool2, n_filters, (3, 3))
    conv3 = residual_block(pool2, n_filters)
    conv3 = residual_block(conv3, n_filters)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    n_filters *= growth_factor
    pool3 = BatchNormalization()(pool3)
    pool3 = convolution_block(pool3, n_filters, (3, 3))
    conv4_0 = residual_block(pool3, n_filters)
    conv4_0 = residual_block(conv4_0, n_filters)
    pool4_0 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_0 = Dropout(droprate)(pool4_0)

    n_filters *= growth_factor
    pool4_0 = BatchNormalization()(pool4_0)
    pool4_0 = convolution_block(pool4_0, n_filters, (3, 3))
    conv4_1 = residual_block(pool4_0, n_filters)
    conv4_1 = residual_block(conv4_1, n_filters)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_1 = Dropout(droprate)(pool4_1)

    n_filters *= growth_factor
    pool4_2 = BatchNormalization()(pool4_1)
    pool4_2 = convolution_block(pool4_2, n_filters, (3, 3))
    conv5 = residual_block(pool4_2, n_filters)
    conv5 = residual_block(conv5, n_filters)
    conv5 = Dropout(droprate)(conv5)
    
    conv5 = concatenate([conv5, lowres_encoder.output])

    n_filters //= growth_factor
    if upconv:
        up6_1 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv5), conv4_1])
    else:
        up6_1 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4_1])
    up6_1 = BatchNormalization()(up6_1)
    up6_1 = convolution_block(up6_1, n_filters, (3, 3))
    conv6_1 = residual_block(up6_1, n_filters)
    conv6_1 = residual_block(conv6_1, n_filters)
    conv6_1 = Dropout(droprate)(conv6_1)

    n_filters //= growth_factor
    if upconv:
        up6_2 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_1), conv4_0])
    else:
        up6_2 = concatenate([UpSampling2D(size=(2, 2))(conv6_1), conv4_0])
    up6_2 = BatchNormalization()(up6_2)
    up6_2 = convolution_block(up6_2, n_filters, (3, 3))
    conv6_2 = residual_block(up6_2, n_filters)
    conv6_2 = residual_block(conv6_2, n_filters)
    conv6_2 = Dropout(droprate)(conv6_2)

    n_filters //= growth_factor
    if upconv:
        up7 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_2), conv3])
    else:
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6_2), conv3])
    up7 = BatchNormalization()(up7)
    up7 = convolution_block(up7, n_filters, (3, 3))
    conv7 = residual_block(up7, n_filters)
    conv7 = residual_block(conv7, n_filters)
    conv7 = Dropout(droprate)(conv7)

    n_filters //= growth_factor
    if upconv:
        up8 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    else:
        up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2])
    up8 = BatchNormalization()(up8)
    up8 = convolution_block(up8, n_filters, (3, 3))
    conv8 = residual_block(up8, n_filters)
    conv8 = residual_block(conv8, n_filters)
    conv8 = Dropout(droprate)(conv8)

    n_filters //= growth_factor
    if upconv:
        up9 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    else:
        up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1])
    up9 = BatchNormalization()(up9)
    up9 = convolution_block(up9, n_filters, (3, 3))
    conv9 = residual_block(up9, n_filters)
    conv9 = residual_block(conv9, n_filters)

    conv10 = Conv2D(n_classes, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs, lowres_encoder.inputs], outputs=conv10)

    return model

In [5]:
model = bi_res_unet(growth_factor=1.5)

2022-06-08 10:00:15.502567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nathan/sandbox/elastix/elastix-5.0.1-linux/lib
2022-06-08 10:00:15.502597: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 10:00:15.502620: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nathan-SBRI): /proc/driver/nvidia/version does not exist


In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_28 (Conv2D)             (None, 224, 224, 32  896         ['input_2[0][0]']                
                                )                                                           

                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 32  0           ['add[0][0]']                    
                                )                                                                 
                                                                                                  
 dropout_6 (Dropout)            (None, 112, 112, 32  0           ['max_pooling2d_5[0][0]']        
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 112, 112, 32  0           ['max_pooling2d[0][0]']          
                                )                                                                 
                                                                                                  
 batch_nor

                                )                                 'batch_normalization_63[0][0]'] 
                                                                                                  
 add_1 (Add)                    (None, 112, 112, 48  0           ['batch_normalization_10[0][0]', 
                                )                                 'batch_normalization_8[0][0]']  
                                                                                                  
 leaky_re_lu_34 (LeakyReLU)     (None, 112, 112, 48  0           ['add_12[0][0]']                 
                                )                                                                 
                                                                                                  
 leaky_re_lu_6 (LeakyReLU)      (None, 112, 112, 48  0           ['add_1[0][0]']                  
                                )                                                                 
          

 leaky_re_lu_36 (LeakyReLU)     (None, 56, 56, 72)   0           ['batch_normalization_71[0][0]'] 
                                                                                                  
 leaky_re_lu_8 (LeakyReLU)      (None, 56, 56, 72)   0           ['batch_normalization_16[0][0]'] 
                                                                                                  
 leaky_re_lu_37 (LeakyReLU)     (None, 56, 56, 72)   0           ['leaky_re_lu_36[0][0]']         
                                                                                                  
 leaky_re_lu_9 (LeakyReLU)      (None, 56, 56, 72)   0           ['leaky_re_lu_8[0][0]']          
                                                                                                  
 batch_normalization_72 (BatchN  (None, 56, 56, 72)  288         ['leaky_re_lu_37[0][0]']         
 ormalization)                                                                                    
          

                                                                                                  
 batch_normalization_24 (BatchN  (None, 56, 56, 72)  288         ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 56, 56, 72)  288         ['add_3[0][0]']                  
 ormalization)                                                                                    
                                                                                                  
 add_15 (Add)                   (None, 56, 56, 72)   0           ['batch_normalization_79[0][0]', 
                                                                  'batch_normalization_77[0][0]'] 
                                                                                                  
 add_4 (Ad

...

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_51 (BatchN  (None, 7, 7, 243)   972         ['leaky_re_lu_26[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 conv2d_54 (Conv2D)             (None, 7, 7, 243)    531684      ['batch_normalization_106[0][0]']
                                                                                                  
 conv2d_26 (Conv2D)             (None, 7, 7, 243)    531684      ['batch_normalization_51[0][0]'] 
                                                                                                  
 batch_normalization_108 (Batch  (None, 7, 7, 243)   972         ['conv2d_54[0][0]']              
 Normaliza

                                                                                                  
 add_22 (Add)                   (None, 14, 14, 162)  0           ['batch_normalization_115[0][0]',
                                                                  'batch_normalization_113[0][0]']
                                                                                                  
 leaky_re_lu_59 (LeakyReLU)     (None, 14, 14, 162)  0           ['add_22[0][0]']                 
                                                                                                  
 batch_normalization_116 (Batch  (None, 14, 14, 162)  648        ['leaky_re_lu_59[0][0]']         
 Normalization)                                                                                   
                                                                                                  
 conv2d_59 (Conv2D)             (None, 14, 14, 162)  236358      ['batch_normalization_116[0][0]']
          

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_127 (Batch  (None, 28, 28, 108)  432        ['add_24[0][0]']                 
 Normalization)                                                                                   
                                                                                                  
 add_25 (Add)                   (None, 28, 28, 108)  0           ['batch_normalization_129[0][0]',
                                                                  'batch_normalization_127[0][0]']
                                                                                                  
 dropout_13 (Dropout)           (None, 28, 28, 108)  0           ['add_25[0][0]']                 
                                                                                                  
 conv2d_tr

 conv2d_71 (Conv2D)             (None, 112, 112, 48  41520       ['batch_normalization_140[0][0]']
                                )                                                                 
                                                                                                  
 batch_normalization_141 (Batch  (None, 112, 112, 48  192        ['conv2d_71[0][0]']              
 Normalization)                 )                                                                 
                                                                                                  
 leaky_re_lu_71 (LeakyReLU)     (None, 112, 112, 48  0           ['batch_normalization_141[0][0]']
                                )                                                                 
                                                                                                  
 leaky_re_lu_72 (LeakyReLU)     (None, 112, 112, 48  0           ['leaky_re_lu_71[0][0]']         
          

                                                                                                  
 leaky_re_lu_77 (LeakyReLU)     (None, 224, 224, 32  0           ['leaky_re_lu_76[0][0]']         
                                )                                                                 
                                                                                                  
 batch_normalization_152 (Batch  (None, 224, 224, 32  128        ['leaky_re_lu_77[0][0]']         
 Normalization)                 )                                                                 
                                                                                                  
 conv2d_77 (Conv2D)             (None, 224, 224, 32  9248        ['batch_normalization_152[0][0]']
                                )                                                                 
                                                                                                  
 batch_nor

In [7]:
plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

In [8]:
import numpy as np

In [16]:
def g(n):
    return np.zeros((224, 224, n))

x = [np.array([g(3)]), np.array([g(3)])]
y = np.array([g(1)])

In [17]:
model.compile(optimizer="adam", loss="mse")

In [18]:
model.fit(x, y)

1/1 [==============================] - 14s 14s/step - loss: 0.2500
